**[Advanced SQL Home Page](https://www.kaggle.com/learn/advanced-sql)**

---


# Introduction

Here, you'll use window functions to answer questions about the [Chicago Taxi Trips](https://www.kaggle.com/chicago/chicago-taxi-trips-bq) dataset.

Before you get started, run the code cell below to set everything up.

In [1]:
# set up feedback system
from learntools.core import binder
binder.bind(globals())
from learntools.sql_advanced.ex2 import *
print("Setup is completed")

Using Kaggle's public dataset BigQuery integration.
Setup is completed


The following code cell fetches the `taxi_trips` table from the `chicago_taxi_trips` dataset. We also preview the first five rows of the table.  You'll use the table to answer the questions below.

In [2]:
# create a "Client" object
from google.cloud import bigquery
client = bigquery.Client()

# construct a reference to the "chicago_taxi_trips" dataset
dataset_ref = client.dataset("chicago_taxi_trips", project="bigquery-public-data")

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

# construct a reference to the "taxi_trips" table
table_ref = dataset_ref.table("taxi_trips")

# API request - fetch the table
table = client.get_table(table_ref)

# preview the first five lines of the table
client.list_rows(table, max_results=5).to_dataframe()

Using Kaggle's public dataset BigQuery integration.


,unique_key,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_census_tract,dropoff_census_tract,pickup_community_area,dropoff_community_area,...,extras,trip_total,payment_type,company,pickup_latitude,pickup_longitude,pickup_location,dropoff_latitude,dropoff_longitude,dropoff_location
0,5ca0f04de0e5c08a254cd55ae0ffd107491f21c9,93ed0b8d2841e33df894a893a95c6287c2b58779b0529d...,2015-05-10 10:30:00+00:00,2015-05-10 11:00:00+00:00,1949,9.0,None,None,14,None,...,0.0,23.05,Cash,Yellow Cab,41.968069,-87.721559,POINT (-87.7215590627 41.968069),None,None,None
1,07ec06d200f3d9199e450d6fb527519f12bbebc8,0fb1abb17fe80c09b7eb931aaa40b4157514f6466a12e2...,2015-05-03 04:15:00+00:00,2015-05-03 04:30:00+00:00,1553,17.3,None,None,14,None,...,1.0,41.92,Credit Card,Yellow Cab,41.968069,-87.721559,POINT (-87.7215590627 41.968069),None,None,None
2,c430712943435faed8227777e0836ed9f4419038,46cb4badd64238b521d799f0aeb48c0214ebcac417ea25...,2015-04-29 21:45:00+00:00,2015-04-29 22:15:00+00:00,1541,5.3,None,None,14,None,...,1.0,15.85,Cash,Yellow Cab,41.968069,-87.721559,POINT (-87.7215590627 41.968069),None,None,None
3,fd026dfd0a58c63707625cf3b3d5cc5f32b33720,c0233d1acdf95f778faf4bcadcff7035dc6bbcab99f07e...,2015-05-02 09:45:00+00:00,2015-05-02 10:00:00+00:00,1509,9.1,None,None,14,None,...,0.0,27.31,Credit Card,Yellow Cab,41.968069,-87.721559,POINT (-87.7215590627 41.968069),None,None,None
4,97b677642b2bdc18e47b5803e81053c6533cf0b0,f705c825446b6687d3b45d5bb04a7a29720306923943f3...,2015-05-09 07:00:00+00:00,2015-05-09 07:15:00+00:00,584,3.1,None,None,14,None,...,0.0,9.45,Cash,Yellow Cab,41.968069,-87.721559,POINT (-87.7215590627 41.968069),None,None,None


# Exercises

## 1. How can you predict the demand for taxis?

Say you work for a taxi company, and you're interested in predicting the demand for taxis.  Towards this goal, you'd like to create a plot that shows a rolling average of the daily number of taxi trips.  Amend the (partial) query below to return a DataFrame with two columns:
- `trip_date` - contains one entry for each date from January 1, 2016, to December 31, 2017.
- `avg_num_trips` - shows the average number of daily trips, calculated over a window including the value for the current date, along with the values for the preceding 15 days and the following 15 days, as long as the days fit within the two-year time frame.  For instance, when calculating the value in this column for January 5, 2016, the window will include the number of trips for the preceding 4 days, the current date, and the following 15 days.

This query is partially completed for you, and you need only write the part that calculates the `avg_num_trips` column.  Note that this query uses a common table expression (CTE); if you need to review how to use CTEs, you're encouraged to check out [this tutorial](https://www.kaggle.com/dansbecker/as-with) in the [Intro to SQL](https://www.kaggle.com/learn/intro-to-sql) micro-course.

In [3]:
# fill in the blank below
avg_num_trips_query = """
WITH trips_by_day AS (
    SELECT
        DATE(trip_start_timestamp) AS trip_date,
        COUNT(*) AS num_trips
    FROM
        `bigquery-public-data.chicago_taxi_trips.taxi_trips`
    WHERE
        DATE(trip_start_timestamp) BETWEEN '2016-01-01' AND '2017-12-31'
    GROUP BY
        trip_date
    ORDER BY
        trip_date
)
SELECT
    trip_date,
    AVG(num_trips) OVER (
        ORDER BY
            trip_date ROWS BETWEEN 15 PRECEDING AND 15 FOLLOWING
    ) AS avg_num_trips
FROM
    trips_by_day
"""

# check your answer
q_1.check()

,trip_date,avg_num_trips
0,2016-01-01,80461.937500
1,2016-01-02,80150.647059
2,2016-01-03,79419.611111
3,2016-01-04,79810.421053
4,2016-01-05,80293.900000


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [4]:
# lines below will give you a hint or solution code
# q_1.hint()
# q_1.solution()

<hr/>

## 2. Can you separate and order trips by community area?

The query below returns a DataFrame with three columns from the table: `pickup_community_area`, `trip_start_timestamp`, and `trip_end_timestamp`.  

Amend the query to return an additional column called `trip_number` which shows the order in which the trips were taken from their respective community areas.  So, the first trip of the day originating from community area 1 should receive a value of 1; the second trip of the day from the same area should receive a value of 2.  Likewise, the first trip of the day from community area 2 should receive a value of 1, and so on.

Note that there are many numbering functions that can be used to solve this problem (depending on how you want to deal with trips that started at the same time from the same community area); to answer this question, please use the **RANK()** function.

In [5]:
# amend the query below
trip_number_query = """
SELECT
    pickup_community_area,
    trip_start_timestamp,
    trip_end_timestamp,
    RANK() OVER (
        PARTITION BY
            pickup_community_area
        ORDER BY
            trip_start_timestamp
    ) AS trip_number
FROM
    `bigquery-public-data.chicago_taxi_trips.taxi_trips`
WHERE
    DATE(trip_start_timestamp) = '2017-05-01'
"""

# check your answer
q_2.check()

,pickup_community_area,trip_start_timestamp,trip_end_timestamp,trip_number
0,71.0,2017-05-01 05:00:00+00:00,2017-05-01 05:30:00+00:00,1
1,71.0,2017-05-01 07:15:00+00:00,2017-05-01 07:30:00+00:00,2
2,71.0,2017-05-01 07:15:00+00:00,2017-05-01 07:15:00+00:00,2
3,71.0,2017-05-01 07:30:00+00:00,2017-05-01 07:30:00+00:00,4
4,71.0,2017-05-01 07:45:00+00:00,2017-05-01 08:15:00+00:00,5


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [6]:
# lines below will give you a hint or solution code
# q_2.hint()
# q_2.solution()

<hr/>

## 3. How much time elapses between trips?

The (partial) query in the code cell below shows, for each trip in the selected time frame, the corresponding `taxi_id`, `trip_start_timestamp`, and `trip_end_timestamp`. 

Your task in this exercise is to edit the query to include an additional `prev_break` column that shows the length of the break (in minutes) that the driver had before each trip started (this corresponds to the time between `trip_start_timestamp` of the current trip and `trip_end_timestamp` of the previous trip).  Partition the calculation by `taxi_id`, and order the results within each partition by `trip_start_timestamp`.

Some sample results are shown below, where all rows correspond to the same driver (or `taxi_id`).  Take the time now to make sure that the values in the `prev_break` column make sense to you!

![first_commands](https://i.imgur.com/qjvQzg8.png)

Note that the first trip of the day for each driver should have a value of **NaN** (not a number) in the `prev_break` column.

In [7]:
# fill in the blanks below
break_time_query = """
SELECT
    taxi_id,
    trip_start_timestamp,
    trip_end_timestamp,
    TIMESTAMP_DIFF(
        trip_start_timestamp,
        LAG(trip_end_timestamp) OVER (
            PARTITION BY
                taxi_id
            ORDER BY
                trip_start_timestamp
        ),
        MINUTE
    ) AS prev_break
FROM
    `bigquery-public-data.chicago_taxi_trips.taxi_trips`
WHERE
    DATE(trip_start_timestamp) = '2017-05-01'
"""

# check your answer
q_3.check()

,taxi_id,trip_start_timestamp,trip_end_timestamp,prev_break
0,0913375eb6e33945aaec0158bd356ee924e3d507323d8f...,2017-05-01 01:30:00+00:00,2017-05-01 02:00:00+00:00,NaN
1,1701aeef6bf3e920b4791572e2ca620701b806d6648b62...,2017-05-01 09:45:00+00:00,2017-05-01 10:45:00+00:00,NaN
2,1701aeef6bf3e920b4791572e2ca620701b806d6648b62...,2017-05-01 11:45:00+00:00,2017-05-01 12:00:00+00:00,60.0
3,1701aeef6bf3e920b4791572e2ca620701b806d6648b62...,2017-05-01 14:45:00+00:00,2017-05-01 15:30:00+00:00,165.0
4,1701aeef6bf3e920b4791572e2ca620701b806d6648b62...,2017-05-01 18:45:00+00:00,2017-05-01 18:45:00+00:00,195.0


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [8]:
# lines below will give you a hint or solution code
q_3.hint()
# q_3.solution()

<IPython.core.display.Javascript object>

<span style="color:#3366cc">Hint:</span> The `TIMESTAMP_DIFF()` function takes three arguments, where the first (`trip_start_timestamp`) and the last (`MINUTE`) are provided for you.  This function provides the time difference (in minutes) of the timestamps in the first two arguments. You need only fill in the second argument, which should use the **LAG()** function to pull the timestamp corresponding to the end of the previous trip (for the same `taxi_id`).

# Keep going

Move on to learn how to query **[nested and repeated data](https://www.kaggle.com/alexisbcook/nested-and-repeated-data)**.

---
**[Advanced SQL Home Page](https://www.kaggle.com/learn/advanced-sql)**





*Have questions or comments? Visit the [Learn Discussion forum](https://www.kaggle.com/learn-forum) to chat with other Learners.*